In [1]:
import time, random, re, urllib.parse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from getpass import getpass
from selenium.webdriver.common.keys import Keys

In [46]:
from functools import wraps

In [2]:
USERNAME = "zh272@georgetown.edu"
PASSWORD = getpass("🔐 Enter LinkedIn Password: ")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Go to LinkedIn login page
driver.get("https://www.linkedin.com/login")
time.sleep(2)

# Enter email and password
driver.find_element(By.ID, "username").send_keys(USERNAME)
driver.find_element(By.ID, "password").send_keys(PASSWORD + Keys.RETURN)
time.sleep(5)

# Verify login
if "feed" in driver.current_url or "jobs" in driver.current_url:
    print("✅ Login successful!")
else:
    print("❌ Login may have failed.")

✅ Login successful!


In [3]:
import itertools

experience_levels = {
"Internship": "1", "Entry level": "2", "Associate": "3",
"Mid-Senior": "4", "Director": "5", "Executive": "6"
}


remote_types = {
"On-site": "1", "Hybrid": "2", "Remote": "3"
}


date_posted = {
"Any time": None,"Past week": "r604800", "Past month": "r2592000"
}


salary_ranges = {
"$40K+": "1", "$60K+": "2", "$80K+": "3", "$100K+": "4", "$120K+": "5"
}


# --- BASE URL ---
base_url = "https://www.linkedin.com/jobs/search/?"


# --- FIXED PARAMETERS ---
fixed_params = {
"keywords": "data center",
"location": "United States",
"sortBy": "R"
}

combinations = list(itertools.product(
experience_levels.items(),
remote_types.items(),
date_posted.items(),
salary_ranges.items()
))

print(f"Generating {len(combinations)} combinations...")

Generating 270 combinations...


In [4]:
import urllib.parse

generated_urls = []

for exp, remote, date, salary in combinations:
    exp_label, exp_val = exp
    remote_label, remote_val = remote
    date_label, date_val = date
    salary_label, salary_val = salary
    query_params = fixed_params.copy()
    query_params.update({
        "f_E": exp_val,
        "f_WT": remote_val,
        "f_TPR": date_val,
        "f_SB2": salary_val
    })
    final_url = base_url + urllib.parse.urlencode(query_params)
    generated_urls.append(final_url)


print(f"Successfully generated {len(generated_urls)} URLs.")

Successfully generated 270 URLs.


In [8]:
driver.get("https://www.linkedin.com/jobs/search/")

In [12]:
job_main = driver.find_element(By.CSS_SELECTOR, "main#main")

In [26]:
job_card_list = job_main.find_elements(By.CSS_SELECTOR, "ul:first-of-type>li.ember-view")

In [28]:
len(job_card_list)
for idx, ele in enumerate(job_card_list):
    print("----- {} -----".format(idx))
    print(ele.text)

----- 0 -----
Avionics Engineering Intern Summer 2026
Avionics Engineering Intern Summer 2026 with verification
Rocket Lab
Long Beach, CA
$28/hr · 401(k), +1 benefit
Viewed
----- 1 -----
University of Cincinnati R&D Engineer Co-op
University of Cincinnati R&D Engineer Co-op with verification
Procter & Gamble
Mason, OH
$29/hr - $50/hr
Promoted
----- 2 -----
Summer 2026 Intern - Machine Learning Engineer
Summer 2026 Intern - Machine Learning Engineer with verification
Workiva
United States (Remote)
401(k) benefit
Promoted
----- 3 -----
2026 Summer Intern, Energy Analyst (Remote)
2026 Summer Intern, Energy Analyst (Remote) with verification
ICF
Reston, VA (Remote)
$34.4K/yr - $58.5K/yr
16 minutes ago
Within the past 24 hours
----- 4 -----
Intern, Air Quality Modeler
Intern, Air Quality Modeler with verification
Ramboll
Fort Collins, CO (On-site)
$22/hr - $25/hr
Promoted
----- 5 -----
IT Intern
IT Intern
Banner Health
Arizona, United States (Remote)
1 hour ago
Within the past 24 hours
----

In [45]:
def safe_text(default=""):
    """装饰器：如果找不到元素就返回 default"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except NoSuchElementException:
                return default
            except AttributeError:
                return default
        return wrapper
    return decorator

In [67]:
@safe_text("")
def get_job_id(ele):
    # ele expected to be a job card element, so the id should located at li>div>div[data-job-id]
    job_id = ele.find_element(By.CSS_SELECTOR, "li>div>div[data-job-id]")
    return job_id.get_attribute("data-job-id")

@safe_text("")
def get_job_name(ele):
    job_name = ele.find_element(By.CSS_SELECTOR, "div.artdeco-entity-lockup__title>a>span")
    return job_name.text.strip()

@safe_text("")
def get_job_subtitle(ele):
    job_subtitle = ele.find_element(By.CSS_SELECTOR, "div.artdeco-entity-lockup__subtitle")
    return job_subtitle.text.strip()

@safe_text("")
def get_job_caption(ele):
    job_caption = ele.find_element(By.CSS_SELECTOR, "div.artdeco-entity-lockup__caption")
    return job_caption.text.strip()

@safe_text("")
def get_job_metadata(ele):
    job_metadata = ele.find_element(By.CSS_SELECTOR, "div.artdeco-entity-lockup__metadata")
    return job_metadata.text.strip()

@safe_text("")
def get_job_url(ele):
    job_url = ele.find_element(By.CSS_SELECTOR, "div.artdeco-entity-lockup__title>a")
    return job_url.get_attribute("href")

In [68]:
temp = job_card_list[5]
print(get_job_id(temp))
print(get_job_name(temp))
print(get_job_subtitle(temp))
print(get_job_caption(temp))
print(get_job_metadata(temp))
print(get_job_url(temp))

4304552661
IT Intern
Banner Health
Arizona, United States (Remote)

https://www.linkedin.com/jobs/view/4304552661/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=OIi1OqXJMurzCpZ1c2NqCQ%3D%3D&trackingId=AuGoe22pWP8HMQ2Or2Hg5A%3D%3D&trk=flagship3_search_srp_jobs


In [ ]:
def extract_job_data(job_card):
    return {
        "job_id": get_job_id(job_card),
        "job_name": get_job_name(job_card),
        "company_name": get_job_subtitle(job_card),
        "job_location": get_job_caption(job_card),
        "job_metadata": get_job_metadata(job_card),
        "job_url": get_job_url(job_card)
    }

In [70]:
extract_job_data(job_card_list[0])

{'job_id': '4304569498',
 'job_name': 'Avionics Engineering Intern Summer 2026',
 'company_name': 'Rocket Lab',
 'job_location': 'Long Beach, CA',
 'job_metadata': '$28/hr · 401(k), +1 benefit',
 'job_url': 'https://www.linkedin.com/jobs/view/4304569498/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=OIi1OqXJMurzCpZ1c2NqCQ%3D%3D&trackingId=OfHrdUa7HEQ%2BUAqR3ieUTQ%3D%3D&trk=flagship3_search_srp_jobs'}

In [53]:
all_job_data = list(map(extract_job_data, job_card_list))

In [54]:
all_job_data

[{'job_id': '4304569498',
  'job_name': 'Avionics Engineering Intern Summer 2026',
  'company_name': 'Rocket Lab',
  'job_location': 'Long Beach, CA',
  'job_metadata': '$28/hr · 401(k), +1 benefit'},
 {'job_id': '4271497616',
  'job_name': 'University of Cincinnati R&D Engineer Co-op',
  'company_name': 'Procter & Gamble',
  'job_location': 'Mason, OH',
  'job_metadata': '$29/hr - $50/hr'},
 {'job_id': '4305583783',
  'job_name': 'Summer 2026 Intern - Machine Learning Engineer',
  'company_name': 'Workiva',
  'job_location': 'United States (Remote)',
  'job_metadata': '401(k) benefit'},
 {'job_id': '4304572433',
  'job_name': '2026 Summer Intern, Energy Analyst (Remote)',
  'company_name': 'ICF',
  'job_location': 'Reston, VA (Remote)',
  'job_metadata': '$34.4K/yr - $58.5K/yr'},
 {'job_id': '4305616666',
  'job_name': 'Intern, Air Quality Modeler',
  'company_name': 'Ramboll',
  'job_location': 'Fort Collins, CO (On-site)',
  'job_metadata': '$22/hr - $25/hr'},
 {'job_id': '430455266

In [65]:
def extract_number_results(ele):
    num_text = ele.find_element(By.CSS_SELECTOR, "header div.jobs-search-results-list__subtitle").text.strip()
    num = re.search(r'(\d[\d,]*)\s+results?', num_text)
    if num:
        return int(num.group(1).replace(",", ""))
    else:   
        return None

In [66]:
extract_number_results(job_main)

3973214

In [76]:
def wait_get_element(driver, selector, timeout=10):
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
        )
        return element
    except TimeoutException:
        print(f"Timeout: Element with selector '{selector}' not found within {timeout} seconds.")
        return None
    
def wait_for_element(driver, selector, timeout=10):
    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
        )
        return True
    except TimeoutException:
        print(f"Timeout: Element with selector '{selector}' not found within {timeout} seconds.")
        return False

In [80]:
def linkedin_common_crawler(driver, url, time_sleep=1, wait_time=10):
    driver.get(url)
    time.sleep(time_sleep)

    job_main = wait_get_element(driver, "main#main", timeout=wait_time)
    if not job_main:
        return None

    # Wait for the first job card to load
    if not wait_for_element(driver, "ul:first-of-type>li.ember-view div.artdeco-entity-lockup__metadata", timeout=wait_time):
        return None
    
    job_card_list = job_main.find_elements(By.CSS_SELECTOR, "ul:first-of-type>li.ember-view")

    all_job_data = list(map(extract_job_data, job_card_list))

    return {
        "url": url,
        "jobs": all_job_data
    }

In [81]:
linkedin_common_crawler(driver, generated_urls[3], time_sleep=0, wait_time=10)

{'url': 'https://www.linkedin.com/jobs/search/?keywords=data+center&location=United+States&sortBy=R&f_E=1&f_WT=1&f_TPR=None&f_SB2=4',
 'jobs': [{'job_id': '4136281761',
   'job_name': 'Applied AI Intern, Deep Learning',
   'company_name': 'Bosch USA',
   'job_location': 'Sunnyvale, CA (On-site)',
   'job_metadata': '$35/hr - $68/hr',
   'job_url': 'https://www.linkedin.com/jobs/view/4136281761/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=fXhux%2B6f%2B%2FDi5PVieKAolA%3D%3D&trackingId=GyGQYJUg24xzLDaWG0n1SA%3D%3D&trk=flagship3_search_srp_jobs'},
  {'job_id': '4214109899',
   'job_name': 'Research Intern (Summer 2026)',
   'company_name': 'camfer',
   'job_location': 'San Francisco, CA (On-site)',
   'job_metadata': '$5,000/yr - $10K/yr',
   'job_url': 'https://www.linkedin.com/jobs/view/4214109899/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=fXhux%2B6f%2B%2FDi5PVieKAolA%3D%3D&trackingId=2C0OaCnhZ0Pk9CiOLZGNQA%3D%3D&trk=flagship3_search_srp_jobs'},
  {'job_id': '4263606346',
   'job_name': 'AI Research I

In [89]:
CONTAINER_SELECTORS = [
    "ul.scaffold-layout__list-container",
    "div.jobs-search-results-list__container",
    "div.jobs-search-two-pane__results",
    "section.two-pane-serp-page__results-list",
]
ITEM_SELECTORS = [
    "li.scaffold-layout__list-item",
    "li[data-occludable-job-id]",
    "ul.jobs-search-results__list > li",
    "div.jobs-search-results-list__list-item",
]

def _sleep(a=0.35, b=0.85): time.sleep(random.uniform(a,b))

def _first_el(root, selectors, by_css=True, timeout=0):
    if timeout:
        for sel in selectors:
            try:
                if by_css:
                    return WebDriverWait(root, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, sel)))
                else:
                    return WebDriverWait(root, timeout).until(EC.presence_of_element_located((By.XPATH, sel)))
            except TimeoutException:
                continue
        return None
    else:
        for sel in selectors:
            try:
                els = root.find_elements(By.CSS_SELECTOR if by_css else By.XPATH, sel)
                if els: return els[0]
            except Exception:
                pass
        return None

def _first_text(root, selectors):
    el = _first_el(root, selectors, by_css=True, timeout=0)
    try: return (el.text or "").strip()
    except: return ""

def _first_attr(root, selectors, attr):
    el = _first_el(root, selectors, by_css=True, timeout=0)
    try: return el.get_attribute(attr) or ""
    except: return ""

def wait_results_loaded(driver, timeout=12):
    for sel in CONTAINER_SELECTORS:
        try:
            WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, sel)))
            return True
        except TimeoutException:
            continue
    return False

def expand_all_in(driver, root=None, max_clicks=8):
    scope = root or driver
    # Both the old & new “See more” buttons
    sels = [
        "button.show-more-less-html__button--more",
        "button.show-more-less-html__button",
        "button[aria-expanded='false']",
    ]
    clicks = 0
    while clicks < max_clicks:
        btn = _first_el(scope, sels, by_css=True, timeout=0)
        if not btn: break
        try:
            btn.click()
        except (ElementClickInterceptedException, StaleElementReferenceException):
            try:
                driver.execute_script("arguments[0].click();", btn)
            except Exception:
                break
        _sleep(0.2, 0.4)
        clicks += 1

def get_title_company(driver):
    title = ""
    company = ""
    try:
        title = WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h1.top-card-layout__title, h1.topcard__title"))
        ).text.strip()
    except TimeoutException:
        pass
    comp_el = _first_el(driver, ["a.topcard__org-name-link", "span.topcard__flavor"], timeout=0)
    if comp_el:
        try: company = comp_el.text.strip()
        except: pass
    return title, company

def get_description_root(driver):
    for sel in ["#job-details", ".jobs-description__content", ".jobs-box__html-content", ".show-more-less-html__markup"]:
        found = driver.find_elements(By.CSS_SELECTOR, sel)
        if found: return found[0]
    return None

def get_salary(driver):
    # Heuristic scan near the top card area first, then global
    areas = [
        ".jobs-unified-top-card", ".job-view-layout", "section.jobs-unified-top-card__primary-description"
    ]
    for area in areas:
        for el in driver.find_elements(By.CSS_SELECTOR, area+" *"):
            try:
                t = (el.text or "").strip()
                if "$" in t or "/hr" in t or "per year" in t or "per hour" in t or "salary" in t.lower():
                    return t
            except: pass
    # fallback
    try:
        t = driver.find_element(By.XPATH, "//*[self::span or self::div][contains(text(),'$') or contains(.,'/hr') or contains(.,'per year') or contains(.,'per hour') or contains(translate(., 'SALARY', 'salary'),'salary')]").text
        return t.strip()
    except Exception:
        return ""

# Split to requested sections
SECTION_HEADERS = [
    ("Responsibilities", ["responsibilities","duties","what you will do","what you’ll do"]),
    ("QualificationsRequired", ["required qualifications","basic qualifications","minimum qualifications","requirements"]),
    ("QualificationsPreferred", ["preferred qualifications","nice to have","preferred"]),
    ("Requirements", ["requirements"]),  # catch-all
]
def split_sections(desc):
    out = {"Description": desc or "", "Responsibilities":"", "QualificationsRequired":"", "QualificationsPreferred":"", "Requirements":""}
    txt = desc or ""
    low = txt.lower()
    hits = []
    for col, keys in SECTION_HEADERS:
        for k in keys:
            i = low.find(k)
            if i != -1:
                hits.append((i, col, k))
                break
    if not hits: return out
    hits.sort(key=lambda x: x[0])
    for idx, (start, col, key) in enumerate(hits):
        end = hits[idx+1][0] if idx+1 < len(hits) else len(txt)
        seg = txt[start:end].strip()
        seg = re.sub(rf"(?is)^{re.escape(key)}\s*:?\s*", "", seg, count=1)
        out[col] = seg.strip()
    return out

In [90]:
def scrape_search_url_to_rows(driver, url, max_pages=40, per_page_limit=None):
    rows = []
    for start in range(0, max_pages*25, 25):
        # build paginated URL
        u = urllib.parse.urlparse(url)
        q = dict(urllib.parse.parse_qsl(u.query, keep_blank_values=True))
        q["start"] = str(start)
        page_url = urllib.parse.urlunparse((u.scheme, u.netloc, u.path, u.params,
                                            urllib.parse.urlencode(q, doseq=True), u.fragment))
        driver.get(page_url)

        if not wait_results_loaded(driver, timeout=12):
            break

        # find job cards
        cards = []
        for sel in ITEM_SELECTORS:
            cards = driver.find_elements(By.CSS_SELECTOR, sel)
            if cards: break
        if not cards:
            break

        list_handle = driver.current_window_handle
        taken = 0

        for job in cards:
            try:
                link_el = _first_el(job, [
                    "a.base-card__full-link", 
                    "a.job-card-container__link", 
                    "a[href*='/jobs/view/']"
                ], timeout=0)
                if not link_el: 
                    continue

                link = link_el.get_attribute("href") or ""
                if not link:
                    continue

                # 🟢 open full job in a new tab
                driver.execute_script("window.open(arguments[0], '_blank');", link)
                WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
                driver.switch_to.window(driver.window_handles[-1])

                WebDriverWait(driver, 12).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".jobs-unified-top-card, h1.top-card-layout__title"))
                )

                expand_all_in(driver, None, max_clicks=6)

                joburl = (driver.current_url or "").split("?")[0]
                title, company = get_title_company(driver)
                desc_root = get_description_root(driver)
                desc = desc_root.text.strip() if desc_root else ""
                salary = get_salary(driver)

                secs = split_sections(desc)

                rows.append({
                    "JobTitle": title,
                    "Company": company,
                    "JobURL": joburl,
                    "SearchURL": url,
                    "Description": secs.get("Description",""),
                    "Responsibilities": secs.get("Responsibilities",""),
                    "QualificationsRequired": secs.get("QualificationsRequired",""),
                    "QualificationsPreferred": secs.get("QualificationsPreferred",""),
                    "Requirements": secs.get("Requirements",""),
                    "Salary": salary,
                })

                # close tab, return to results
                driver.close()
                driver.switch_to.window(list_handle)
                _sleep()

                taken += 1
                if per_page_limit and taken >= per_page_limit:
                    break

            except Exception:
                try:
                    if driver.current_window_handle != list_handle:
                        driver.close()
                        driver.switch_to.window(list_handle)
                except:
                    pass
                continue

        if len(cards) < 10:
            break
        _sleep(0.9, 1.6)
    return rows


In [91]:
def scrape_and_collect(driver, urls, max_pages=40, per_page_limit=None):
    """
    Scrape LinkedIn jobs from a list of search URLs.
    Returns a Pandas DataFrame with the results.
    (CSV writing removed — you can export later.)
    """
    all_rows = []
    for u in urls:
        rows = scrape_search_url_to_rows(
            driver, u, max_pages=max_pages, per_page_limit=per_page_limit
        )
        all_rows.extend(rows)
        _sleep(1.0, 2.0)

    df = pd.DataFrame(all_rows)
    if "JobURL" in df.columns:
        df = df.drop_duplicates(subset=["JobURL"], keep="first")
    print(f"Collected {len(df)} rows from {len(urls)} search URLs")
    return df

In [92]:
df = scrape_and_collect(driver, [generated_urls[0]], max_pages=1)

Collected 0 rows from 1 search URLs


In [93]:
df

""
